In [1]:
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
import pandas as pd
import emoji
import numpy as np
import math
import statistics
from scipy.stats import skew,kurtosis
import scipy
import string
from datetime import datetime, timedelta
from collections import Counter
from scipy import stats
import json
import re
import ast
import time

In [2]:
#Read file that contains all tweets
tweets = pd.read_csv(r"Final_Data_v3_1.csv")

In [3]:
#Group tweets according to user id and remove users with less than 2 tweets
tweets = tweets.groupby('user.id_str').filter(lambda x : len(x)>1)
tweets.reset_index(inplace = True, drop = True)

In [41]:
tweets

,Unnamed: 0,source,created_at,id_str,text,in_reply_to_user_id_str,retweet_count,favorite_count,favorited,retweeted,...,number_of_emojis,text_length_words,text_length_characters,text_puncts,url_number,mention_number,hashtag_number,Datetime,Hour,Weekday
0,0,Twitter for Android,Thu Dec 02 11:33:44 +0000 2021,1466369992438566912,RT @Boboj29: La Procura di Torino e' alla rice...,NaN,14,0,False,False,...,0,25,140,4,0,1,0,2021-12-02 11:33:44+00:00,11,3
1,1,Twitter for Android,Thu Dec 02 11:33:35 +0000 2021,1466369952764641283,RT @mgmaglie: Intervista al prefetto di Pesaro...,NaN,457,0,False,False,...,0,20,140,7,0,1,0,2021-12-02 11:33:35+00:00,11,3
2,2,Twitter for Android,Thu Dec 02 11:33:06 +0000 2021,1466369831050043399,"RT @Djnc78: ""I giocatori della #Juventus sono ...",NaN,44,0,False,False,...,0,18,129,10,0,1,1,2021-12-02 11:33:06+00:00,11,3
3,3,Twitter for Android,Thu Dec 02 11:32:44 +0000 2021,1466369737940733953,RT @zilianisky66: @WindTreOfficial dopo 20 ' i...,NaN,1,0,False,False,...,0,22,139,7,0,2,0,2021-12-02 11:32:44+00:00,11,3
4,4,Twitter for Android,Thu Dec 02 11:32:20 +0000 2021,1466369640649707521,"@WindTreOfficial dopo 20 ' in,attesa al telefo...",8.564342e+17,1,1,False,False,...,0,37,218,15,0,1,0,2021-12-02 11:32:20+00:00,11,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476408,476411,Twitter for iPhone,Sun Jan 30 13:02:00 +0000 2022,1487773087248519168,RT @matsulie_mu: 【✨ライブ配信終了✨】\n#スカパラ オンラインライブ「P...,NaN,107,0,False,False,...,3,5,140,5,0,1,1,2022-01-30 13:02:00+00:00,13,6
476409,476412,Twitter for iPhone,Sun Jan 30 08:13:27 +0000 2022,1487700470755852291,RT @Yahoo_GYAO: ＼#フォーリミ がテーブルクロス引きチャレンジ?!／\n#G...,NaN,184,0,False,False,...,4,7,140,13,1,1,2,2022-01-30 08:13:27+00:00,8,6
476410,476413,Twitter for iPhone,Sun Jan 30 04:42:07 +0000 2022,1487647287819919362,【今夜！ゲスト出演！】\n東京スカパラダイスオーケストラの配信ライブに、GENがゲスト出演🕺...,NaN,47,824,False,False,...,3,3,128,10,1,0,0,2022-01-30 04:42:07+00:00,4,6
476411,476414,Twitter for iPhone,Mon Jan 24 09:34:12 +0000 2022,1485546466634792960,RT @Yahoo_GYAO: LIVE HOLIC extra 2022 最速無料配信スタ...,NaN,122,0,False,False,...,3,11,140,9,0,2,2,2022-01-24 09:34:12+00:00,9,0


In [4]:
#Method that extracts emojis from a string

def extract_emojis(s):
  return ''.join(c for c in s if c in emoji.UNICODE_EMOJI['en'])

In [5]:
#Method that extracts statistics from a vector
#More specifically, it calculates MIN, MAX, MEAN, MEDIAN, Standard Deviation, skewness, kyrtosis, entropy and quantiles 0.25 and 0.75

def calc_vector_stats(vectors, column_names):
    
    df_final = pd.DataFrame(columns = column_names)
    
    for key in vectors:
        a = vectors[key]
        MIN = min(a)
        MAX = max(a)
        MEAN = statistics.mean(a)
        MEDIAN = statistics.median(a)
        STD = np.std(a)
        SKEW = skew(a)
        KYRT = kurtosis(a)
        pd_series = pd.Series(a)
        counts = pd_series.value_counts()
        ENTROPY = scipy.stats.entropy(counts)
        q1 = np.percentile(a, 25)
        q3 = p = np.percentile(a, 75) #Q3
        x = [key, MIN, MAX, MEAN, MEDIAN, STD, SKEW, KYRT, ENTROPY, q1, q3]     
        df_final.loc[-1] = x
        df_final.index +=1
        df_final = df_final.sort_index()
        
    return df_final

In [6]:
def call_both(column_name, previous_df):
    vector = function_column_adder(column_name)
    a = merge_dataframes(vector,column_name,previous_df)
    return a

def function_column_adder(column_name):
    users  = tweets['user.id_str'].value_counts()
    vectors = {}
    for ID in users.index:
        new_df = tweets[tweets['user.id_str'] == ID]
        new_df2 = new_df[column_name]
        a = new_df2.values.tolist()
        vectors.update({ID:a})
    return vectors

def merge_dataframes (vectors,column_name,previous_df):
    column_names =  ["User_id", "Min_"+column_name, "Max_"+column_name,"Mean_"+column_name, "Median_"+column_name, "Std_"+column_name, "Skew_"+column_name, "Kyrt_"+column_name, "Entropy_"+column_name, "Q1_"+column_name, "Q3_"+column_name]
    to_be_added = calc_vector_stats(vectors,column_names)
    return pd.merge(previous_df, to_be_added , on='User_id')

In [7]:
#Method that calculates time between consecutive tweets created by each user

def get_time_between_tweets(tweets):
    gaplist = {}
    users  = tweets['user.id_str'].value_counts()
    vectors_text_length = {}
    for ID in users.index:
        gap = []
        new_df = tweets[tweets['user.id_str'] == ID]
        new_df.reset_index(inplace = True, drop = True)
        #print(ID)
        for i in range(0, len(new_df) - 1):
            #Find the time that the current tweet was created
            first = datetime.strptime(new_df['created_at'][i],'%a %b %d %H:%M:%S +0000 %Y')
            #Find the time that the next consecutive tweet was created
            second = datetime.strptime(new_df['created_at'][i + 1],'%a %b %d %H:%M:%S +0000 %Y')
            #Store the difference
            gap.append((first - second).seconds)
        gaplist.update({ID:gap})
    return gaplist

In [8]:
#Method that calculates the number of tweets each hour of day

def tweets_per_hour(tweets):
    users  = tweets['user.id_str'].value_counts()
    vectors = {}
    for ID in users.index:
        a = np.zeros(24)
        new_df = tweets[tweets['user.id_str']== ID]
        new_df2 = new_df['Hour'].value_counts()
        for times,value in zip(new_df2.index,new_df2):
            a[times] = value
        vectors.update({ID:a})
    return vectors

In [9]:
#Method that calculates the number of tweets each weekday

def tweets_per_weekday(tweets):
    testdf  = tweets['user.id_str'].value_counts()
    vectors = {}
    for ID in testdf.index:
        a = np.zeros(7)
        new_df = tweets[tweets['user.id_str']== ID]
        new_df2 = new_df['Weekday'].value_counts()
        for times,value in zip(new_df2.index,new_df2):
            a[times] = value
        vectors.update({ID:a})
    return vectors

## Sentiment

In [10]:
start = time.time()
#Number of emoticons in each tweet and statistics
tweets["emoji"] = tweets["text"].map(lambda x:extract_emojis(x))
tweets["number_of_emojis"] = tweets["emoji"].map(lambda x:len(x))
column_names =  ["User_id", "Min_emo", "Max_emo","Mean_emo", "Median_emo", "Std_emo", "Skew_emo", "Kyrt_emo", "Entropy_emo", "Q1_emo", "Q3_emo"]

testdf  = tweets['user.id_str'].value_counts()
vectors_emoji = {}
ratio = {}
for ID in testdf.index:
    new_df = tweets[tweets['user.id_str']== ID]
    new_df2 = new_df['number_of_emojis']
    a = new_df2.values.tolist()
    ratio.update({ID:((len(a)-a.count(0))/len(a))})
    end1 = time.time()
    vectors_emoji.update({ID:a})
df_emoji_stats = calc_vector_stats(vectors_emoji,column_names)
end = time.time()
print(end - start)
print(end1 - start)

31.298998594284058
27.08483862876892


In [11]:
ration = pd.DataFrame.from_dict(ratio, orient='index')
ration.reset_index(inplace = True)
ration.rename(columns = {'index':'User_id', 0:'Emoji_Ratio'}, inplace = True)
df_emoji_stats = pd.merge(df_emoji_stats, ration , on='User_id')

## Content

In [12]:
start = time.time()
#Number of words in a Tweet and statistics
tweets["text_length_words"] = tweets["text"].map(lambda x:len(x.split(" ")))
features = call_both("text_length_words", df_emoji_stats)
end = time.time()
print(end - start)

6.204496383666992


In [13]:
start = time.time()
#Number of charactes in a Tweet and statistics
tweets["text_length_characters"] = tweets["text"].map(lambda x:len(x))
features = call_both("text_length_characters", features)
end = time.time()
print(end - start)

5.826003313064575


In [14]:
start = time.time()
#Number of punctuation marks in a Tweet and statistics
count = lambda l1,l2: sum([1 for x in l1 if x in l2])
tweets["text_puncts"] = tweets["text"].map(lambda x :count(x,set(string.punctuation)))
features = call_both("text_puncts", features)
end = time.time()
print(end - start)

7.7313196659088135


In [15]:
start = time.time()
#Number of URLs in a Tweet and statistics
tweets['url_number'] = tweets['entities.urls'].apply(lambda x :len(ast.literal_eval(x)) if (len(x)>2)  else 0 )
features = call_both('url_number', features)
end = time.time()
print(end - start)

7.81576681137085


In [16]:
start = time.time()
#Number of mentions in a Tweet and statistics
tweets['mention_number'] = tweets['entities.user_mentions'].apply(lambda x :len(ast.literal_eval(x)) if (len(x)>2)  else 0 )
features = call_both('mention_number', features)
end = time.time()
print(end - start)

14.700809478759766


In [17]:
start = time.time()
#Number of hashtags in a Tweet and statistics
#new_df = tweets[tweets['entities.hashtags'].map(lambda d: len(d)) > 0]
tweets['hashtag_number'] = tweets['entities.hashtags'].apply(lambda x :len(ast.literal_eval(x)) if (len(x)>2)  else 0 )
features = call_both('hashtag_number', features)
end = time.time()
print(end - start)

8.426362752914429


## Temporal

In [18]:
start = time.time()
#Time between two consecutive posts (both tweets and retweets) and statistics
times = get_time_between_tweets(tweets)
features = merge_dataframes(times, "time", features)
end = time.time()
print(end - start)

17.319398880004883


In [19]:
start = time.time()
#Time between two consecutive retweets and statistics
re_tweets = tweets[~tweets['retweeted_status.id_str'].isnull()]
re_tweets.reset_index(inplace = True,drop = True)
re_tweets = re_tweets.groupby('user.id_str').filter(lambda x : len(x)>1)
re_tweets.reset_index(inplace = True,drop = True)

times = get_time_between_tweets(re_tweets)
features = merge_dataframes(times, "time_rt", features)
end = time.time()
print(end - start)

10.178314208984375


In [20]:
start = time.time()
#Time between two consecutive tweets and statistics
normal_tweets = tweets[tweets['retweeted_status.id_str'].isnull()]
normal_tweets.reset_index(inplace = True,drop = True)
normal_tweets = normal_tweets.groupby('user.id_str').filter(lambda x : len(x)>1)
normal_tweets.reset_index(inplace = True,drop = True)

times = get_time_between_tweets(normal_tweets)
features = merge_dataframes(times, "time_normal", features)
end = time.time()
print(end - start)

11.658301830291748


In [21]:
start = time.time()
#Time between two consecutive mentions and statistics
new_df = tweets[tweets['entities.user_mentions'].map(lambda d: len(d)) > 2]
new_df.reset_index(inplace= True,drop = True)
new_df = new_df.groupby('user.id_str').filter(lambda x : len(x)>1)
times = get_time_between_tweets(new_df)
features = merge_dataframes(times, "time_mentions", features)
end = time.time()
print(end - start)

12.9547278881073


In [22]:
start = time.time()
#Tweets per Hour

#Convert 'created_at' feature to datetime
tweets['Datetime'] = pd.to_datetime(tweets['created_at'], format= "%a %b %d %H:%M:%S %z %Y")
tweets['Hour'] = tweets['Datetime'].apply(lambda x: x.hour)
hour = tweets_per_hour(tweets)
features = merge_dataframes(hour,'hour', features)
end = time.time()
print(end - start)

9.996447086334229


In [23]:
start = time.time()
#Tweets per Weekday

tweets['Weekday'] = tweets['Datetime'].apply(lambda x: x.weekday())
day = tweets_per_weekday(tweets)
features = merge_dataframes(day,'day',features)
end = time.time()
print(end - start)

7.344275712966919


## User

In [24]:
users1 = pd.read_json(r"gilani-2017_tweets.json")

user_features1 = pd.DataFrame(users1['user'].values.tolist(),
                             index=users1.index)
user_entities1 = pd.DataFrame(user_features1["entities"].values.tolist(),
                             index=user_features1.index)
user_url1 =  pd.DataFrame(user_entities1["url"].values.tolist(), 
                         index=user_entities1.index)
merged1 = pd.concat([user_features1,user_url1], axis=1)

In [25]:
users2 = pd.read_json(r"cresci-rtbust-2019_tweets.json")

user_features2 = pd.DataFrame(users2['user'].values.tolist(),
                             index=users2.index)
user_entities2 = pd.DataFrame(user_features2["entities"].values.tolist(),
                             index=user_features2.index)
user_url2 =  pd.DataFrame(user_entities2["url"].values.tolist(), 
                         index=user_entities2.index)

merged2 = pd.concat([user_features2,user_url2], axis=1)
merged = pd.concat([merged1, merged2], axis = 0)
merged.reset_index(inplace = True, drop = True)

In [26]:
start = time.time()
#Number of characters in user name
name_char_len = []

for i in range(len(merged)):
    name_char_len.append(len(merged['name'][i]))

merged['name_char_len'] = pd.DataFrame(name_char_len)
end = time.time()
print(end - start)

start = time.time()
#Number of characters in screen name
screen_name_char_len = []

for i in range(len(merged)):
    screen_name_char_len.append(len(merged['screen_name'][i]))
    
merged['screen_name_char_len'] = pd.DataFrame(screen_name_char_len)
end = time.time()
print(end - start)

start = time.time()
#Number of words in user name
name_words_num = []

for i in range(len(merged)):
    name_words_num.append(len(merged['name'][i].split()))
    
merged['name_words_num'] = pd.DataFrame(name_words_num)
end = time.time()
print(end - start)

0.012002706527709961
0.011002302169799805
0.012005090713500977


In [1]:
start = time.time()
from textdistance import levenshtein
merged  = merged.assign(distance=[*map(levenshtein.distance, merged.name, merged.screen_name)])
end = time.time()
print(end - start)

In [28]:
merged['Datetime'] = pd.to_datetime(merged['created_at'], format= "%a %b %d %H:%M:%S %z %Y" )

In [29]:
start = time.time()
#Calculate how many days have passed after the creation of the account
from datetime import date
dates =[]
#s1 = now.strftime("%m/%d/%Y, %H:%M:%S")
# mm/dd/YY H:M:S format
#print("s1:", s1)
for i in range(len(merged)):
    year = merged['Datetime'][i].year
    month = merged['Datetime'][i].month
    day = merged['Datetime'][i].day
    d0 = date(year, month, day)
    d1 = date(2022, 5, 14)
    delta = d1 - d0
    dates.append(delta.days)
    
# Account Age    
merged['Date_Delta'] = pd.DataFrame(dates)
end = time.time()
print(end - start)

0.05846261978149414


In [30]:
start = time.time()
#Number of followers per day
merged['Followers_Per_Day'] = merged['followers_count'] / merged['Date_Delta']
end = time.time()
print(end - start)

start = time.time()
#Number of friends per day
merged['Friends_Per_Day'] = merged['friends_count'] / merged['Date_Delta']
end = time.time()
print(end - start)

#Number of favourites per day
start = time.time()
merged['Favourites_Per_Day'] = merged['favourites_count'] / merged['Date_Delta']
end = time.time()
print(end - start)

0.001001119613647461
0.0009999275207519531
0.0


In [31]:
start = time.time()
#User's timezone
merged["timezone"] = merged["Datetime"].map(lambda x : x.strftime("%z"))
end = time.time()
print(end - start)

0.019004106521606445


In [32]:
start = time.time()
#Followers-Friends Ratio
merged["Followers_Friends_Ratio"] = merged["followers_count"]/merged["friends_count"]
end = time.time()
print(end - start)

start = time.time()
#If uses default image
merged["Uses_default_profile_image_url"] = np.where(merged["profile_image_url"] == "http://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png",True,False)
end = time.time()
print(end - start)

start = time.time()
#If uses unique description
merged["not_using_unique_description"] = merged['description'].duplicated(keep=False)
end = time.time()
print(end - start)

0.0010001659393310547
0.0010001659393310547
0.0010006427764892578


In [33]:
merged = merged[['id_str',
                                'protected',
                          'followers_count',
                            'friends_count',
                             'listed_count',
                         'favourites_count',
                              'geo_enabled',
                                 'verified',
                           'statuses_count',
                                     'lang',
                   'is_translation_enabled',
                  'profile_background_tile',
                     'has_extended_profile',
                          'default_profile',
                                'following',
                      'follow_request_sent',
                          'translator_type',
                            'name_char_len',
                     'screen_name_char_len',
                           'name_words_num',
                                 'distance',
                                 'Datetime',
                               'Date_Delta',
                        'Followers_Per_Day',
                          'Friends_Per_Day',
                       'Favourites_Per_Day',
                                 'timezone',
                  'Followers_Friends_Ratio',
           'Uses_default_profile_image_url',
             'not_using_unique_description']]

In [34]:
merged.rename(columns = {"id_str":"User_id"}, inplace = True)

In [35]:
merged = merged.astype({"User_id": float})
type(merged["User_id"][0])

numpy.float64

In [36]:
type(features["User_id"][0])

numpy.float64

In [37]:
merged

,User_id,protected,followers_count,friends_count,listed_count,favourites_count,geo_enabled,verified,statuses_count,lang,...,distance,Datetime,Date_Delta,Followers_Per_Day,Friends_Per_Day,Favourites_Per_Day,timezone,Followers_Friends_Ratio,Uses_default_profile_image_url,not_using_unique_description
0,6.135120e+08,False,121500,346,532,35894,True,False,21246,en,...,2,2012-06-20 15:07:36+00:00,3615,33.609959,0.095712,9.929184,+0000,351.156069,False,False
1,1.926398e+07,False,3227215,1946,9039,16134,True,True,45791,en,...,16,2009-01-20 23:15:20+00:00,4862,663.762855,0.400247,3.318387,+0000,1658.383864,False,False
2,1.624329e+09,False,864968,767106,1371,23,True,False,875763,ar,...,10,2013-07-27 01:44:55+00:00,3213,269.208839,238.750700,0.007158,+0000,1.127573,False,True
3,5.506100e+08,False,951,275,9,22147,True,False,88862,en,...,3,2012-04-11 01:52:30+00:00,3685,0.258073,0.074627,6.010041,+0000,3.458182,False,False
4,1.497146e+08,False,1412,623,12,5582,True,False,142073,en,...,11,2010-05-30 01:32:39+00:00,4367,0.323334,0.142661,1.278223,+0000,2.266453,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3182,2.982573e+09,False,152,46,0,1002,True,False,2134,it,...,12,2015-01-17 14:48:27+00:00,2674,0.056844,0.017203,0.374720,+0000,3.304348,False,True
3183,8.256987e+17,False,301,863,0,546,False,False,3938,it,...,2,2017-01-29 13:34:35+00:00,1931,0.155878,0.446919,0.282755,+0000,0.348783,False,False
3184,7.513717e+17,False,35,33,26,6953,False,False,6858,it,...,0,2016-07-08 11:05:47+00:00,2136,0.016386,0.015449,3.255150,+0000,1.060606,False,True
3185,7.267557e+17,False,134,155,1,13631,True,False,12439,it,...,10,2016-05-01 12:50:44+00:00,2204,0.060799,0.070327,6.184664,+0000,0.864516,False,False


In [38]:
all_features = merged.merge(features, on = "User_id")

In [39]:
all_features.to_csv("all_features.csv")

In [40]:
all_features

,User_id,protected,followers_count,friends_count,listed_count,favourites_count,geo_enabled,verified,statuses_count,lang,...,Min_day,Max_day,Mean_day,Median_day,Std_day,Skew_day,Kyrt_day,Entropy_day,Q1_day,Q3_day
0,6.135120e+08,False,121500,346,532,35894,True,False,21246,en,...,20.0,41.0,28.714286,30.0,7.477858,0.284134,-1.310917,1.945910,21.5,33.5
1,1.926398e+07,False,3227215,1946,9039,16134,True,True,45791,en,...,5.0,44.0,28.714286,30.0,11.106167,-0.978557,0.509374,1.945910,27.5,33.5
2,5.506100e+08,False,951,275,9,22147,True,False,88862,en,...,13.0,45.0,28.714286,30.0,10.402119,-0.073946,-1.112115,1.945910,21.0,35.5
3,1.497146e+08,False,1412,623,12,5582,True,False,142073,en,...,9.0,43.0,28.714286,31.0,12.791898,-0.493307,-1.219980,1.747868,20.5,38.5
4,7.694399e+07,False,1615,182,27,34629,True,False,85419,es,...,10.0,48.0,28.714286,31.0,12.532407,-0.011864,-1.264797,1.945910,18.5,37.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2098,4.770148e+08,False,12772,13140,42,44510,True,False,89713,en,...,19.0,48.0,28.714286,25.0,9.145089,1.070635,0.101125,1.945910,22.5,32.0
2099,2.982573e+09,False,152,46,0,1002,True,False,2134,it,...,1.0,157.0,28.714286,2.0,53.611604,1.867110,1.729060,1.153742,2.0,18.5
2100,8.256987e+17,False,301,863,0,546,False,False,3938,it,...,0.0,113.0,28.714286,4.0,41.664354,1.163739,-0.348898,1.549826,2.0,40.0
2101,7.267557e+17,False,134,155,1,13631,True,False,12439,it,...,13.0,53.0,28.714286,31.0,13.530012,0.405850,-0.924289,1.747868,16.5,35.5


## Network - Social Neighborhood

In [45]:
import neo4j
from py2neo import Graph, Node, Relationship
from py2neo.matching import *
from py2neo import Graph
from py2neo.bulk import create_relationships
from itertools import islice
import ast
from py2neo.bulk import create_nodes

In [46]:
features = pd.read_csv('all_features.csv')

In [47]:
features

,Unnamed: 0,User_id,protected,followers_count,friends_count,listed_count,favourites_count,geo_enabled,verified,statuses_count,...,Min_day,Max_day,Mean_day,Median_day,Std_day,Skew_day,Kyrt_day,Entropy_day,Q1_day,Q3_day
0,0,6.135120e+08,False,121500,346,532,35894,True,False,21246,...,20.0,41.0,28.714286,30.0,7.477858,0.284134,-1.310917,1.945910,21.5,33.5
1,1,1.926398e+07,False,3227215,1946,9039,16134,True,True,45791,...,5.0,44.0,28.714286,30.0,11.106167,-0.978557,0.509374,1.945910,27.5,33.5
2,2,5.506100e+08,False,951,275,9,22147,True,False,88862,...,13.0,45.0,28.714286,30.0,10.402119,-0.073946,-1.112115,1.945910,21.0,35.5
3,3,1.497146e+08,False,1412,623,12,5582,True,False,142073,...,9.0,43.0,28.714286,31.0,12.791898,-0.493307,-1.219980,1.747868,20.5,38.5
4,4,7.694399e+07,False,1615,182,27,34629,True,False,85419,...,10.0,48.0,28.714286,31.0,12.532407,-0.011864,-1.264797,1.945910,18.5,37.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2098,2098,4.770148e+08,False,12772,13140,42,44510,True,False,89713,...,19.0,48.0,28.714286,25.0,9.145089,1.070635,0.101125,1.945910,22.5,32.0
2099,2099,2.982573e+09,False,152,46,0,1002,True,False,2134,...,1.0,157.0,28.714286,2.0,53.611604,1.867110,1.729060,1.153742,2.0,18.5
2100,2100,8.256987e+17,False,301,863,0,546,False,False,3938,...,0.0,113.0,28.714286,4.0,41.664354,1.163739,-0.348898,1.549826,2.0,40.0
2101,2101,7.267557e+17,False,134,155,1,13631,True,False,12439,...,13.0,53.0,28.714286,31.0,13.530012,0.405850,-0.924289,1.747868,16.5,35.5


In [48]:
tweets['entities.user_mentions'] = tweets['entities.user_mentions'].apply(lambda x :ast.literal_eval(x))

In [49]:
tweets = tweets.drop_duplicates(subset=['id_str']).reset_index(drop=True)
tweets = tweets.dropna(subset = ['id_str'])
g = Graph(password='0000')
graph = Graph(password='0000')

In [50]:
tweets.columns

Index(['Unnamed: 0', 'source', 'created_at', 'id_str', 'text',
       'in_reply_to_user_id_str', 'retweet_count', 'favorite_count',
       'favorited', 'retweeted', 'entities.hashtags', 'entities.symbols',
       'entities.user_mentions', 'entities.urls', 'user.id', 'user.id_str',
       'retweeted_status.created_at', 'retweeted_status.id',
       'retweeted_status.id_str', 'text.1', 'emoji', 'number_of_emojis',
       'text_length_words', 'text_length_characters', 'text_puncts',
       'url_number', 'mention_number', 'hashtag_number', 'Datetime', 'Hour',
       'Weekday'],
      dtype='object')

In [51]:
#Remove users that have zero mentions
mask = tweets[tweets['entities.user_mentions'].str.len() > 0]
mask.reset_index(inplace = True, drop = True)

In [52]:
original_tweets = mask[mask['retweeted_status.id_str'].isnull()]
retweeted_tweets = mask.dropna(subset = ['retweeted_status.id_str'])

In [53]:
original_tweets = original_tweets[['id_str',
                                   'user.id_str',
                                   'entities.user_mentions']]
original_tweets = original_tweets.reset_index(drop = True)
retweeted_tweets = retweeted_tweets[['id_str',
                                     'user.id_str',
                                     'retweeted_status.id_str',
                                     'entities.user_mentions']]
retweeted_tweets = retweeted_tweets.reset_index(drop = True)

In [54]:
original_author = {}
for t in retweeted_tweets.index:
    user_mentioned_json = retweeted_tweets["entities.user_mentions"][t]
    user_mentioned_loaded= json.loads(json.dumps(user_mentioned_json))
    i=0
    for screen_names in range(len(user_mentioned_loaded)):
        if i==0 :
            user_name_mentioned = user_mentioned_loaded[screen_names]['screen_name'] 
            user_mentioned_id = user_mentioned_loaded[screen_names]['id_str']
            original_author.update({retweeted_tweets['id_str'][t]:(user_mentioned_id,user_name_mentioned)})
            break
#we break as soon as we find the first one cause thats the only one we need

In [55]:
Original_authors = pd.DataFrame(original_author.values(), columns=['user.id_str', 'user.screen_name'])
retweeted_tweets.drop('id_str', axis=1, inplace=True)
#retweeted_tweets.drop('entities.user_mentions', axis=1, inplace=True)
retweeted_tweets.drop('user.id_str', axis=1, inplace=True)
retweeted_tweets = pd.concat([retweeted_tweets, Original_authors], axis=1,join='inner')
retweeted_tweets = retweeted_tweets.rename(columns={'retweeted_status.id_str': 'id_str',
                                                    'retweeted_status.entities.user_mentions':'entities.user_mentions'})

In [56]:
original_final = pd.concat([original_tweets,retweeted_tweets],ignore_index=False)
original_final.reset_index(drop=True,inplace=True)
counts = original_final
original_final = original_final.drop_duplicates(subset=['id_str'],keep = 'first').reset_index(drop=True)
original_final.drop('user.screen_name', axis=1, inplace=True)
original_final.drop('entities.user_mentions', axis=1, inplace=True)

In [57]:
unique_tweets = original_final.to_dict('records')
stream = iter(unique_tweets)
batch_size = 100000000
batch = islice(stream, batch_size)
create_nodes(graph.auto(), batch, labels={"Tweet"})

In [58]:
mentioned_users_temp = {}
for t in mask.index:
    user_mentioned_json = mask["entities.user_mentions"][t]
    user_mentioned_loaded= json.loads(json.dumps(user_mentioned_json))
    for screen_names in range(len(user_mentioned_loaded)):    
        user_name_mentioned = user_mentioned_loaded[screen_names]['screen_name'] 
        user_mentioned_id = user_mentioned_loaded[screen_names]['id_str'] 
        temp_dict = {user_mentioned_id:user_name_mentioned}
        mentioned_users_temp.update(temp_dict)
#To make the loop faster  we use a dictionary and then we turn the dictionary in a dataframe.
mentioned_users = pd.DataFrame(list(mentioned_users_temp.items()),columns = ['user.id_str','user.screen_name']) 

In [59]:
users = mask[['user.id_str']]#tweets[['user.id_str','user.screen_name','user.created_at','user.followers_count']]
users_rt = retweeted_tweets[['user.id_str']]
original_tweets[['user.id_str']]
final_users = pd.concat([users,users_rt, mentioned_users],ignore_index=False)
final_users = final_users.drop_duplicates(subset=['user.id_str'],keep = 'first').reset_index(drop=True)
#final_users.rename(columns={'user.id_str':''})
unique_users = final_users.to_dict('records')

In [60]:
stream = iter(unique_users)
batch_size = 50000000000
batch = islice(stream, batch_size)
create_nodes(graph.auto(), batch, labels={"User"})

In [61]:
matcher=NodeMatcher(g)
nodes=matcher.match("User")
master_user_dict = {}
#here we create master_user_dict while parsing the nodes
#master_user_dict is a dictionary that has a key in the form of the id of the user and value in the form of the id of the node that represents the user
for node in nodes:
    nodeid = node.identity
    userid = (list(node.values())[0])
    master_user_dict.update({userid:nodeid})

In [62]:
mentioned_users
master_user_dict
mentions = mask[['user.id_str','entities.user_mentions']]

In [63]:
test_list = []
temp_dict = {}
for i in mentions.index:
#for each user we refer to the dictionaries we have created before.
# Reform the json as we did before
    current_user = mentions['user.id_str'][i]
    a = master_user_dict[current_user]
    user_mentioned_json = mentions['entities.user_mentions'][i]
    user_mentioned_loaded= json.loads(json.dumps(user_mentioned_json))
    for screen_names in range(len(user_mentioned_loaded)):
#for each user we refer to master_user_dict to get the node id 
#this way we get both users node id so we create the tuple
        user_mentioned_id = user_mentioned_loaded[screen_names]['id_str']
        b = master_user_dict[user_mentioned_id]
        temp_tuple = (a,temp_dict,b)
        test_list.append(temp_tuple)

In [64]:
x = int(len(test_list)/2)
lista  = test_list[:x]
listb = test_list[x:]

In [65]:
create_relationships(g.auto(), lista, "Mention")
create_relationships(g.auto(), listb, "Mention")

In [66]:
#Store the number of times each user mentions someone or is mentioned by someone
#Store the number of neighbors of each user

features["mentions"] = 0 
features["neighbors"] = 0 
i = -1
for user_id in features["User_id"]:
    i = i + 1
    a = graph.run(f'MATCH (n1:User)-[r]-(n2:User) WHERE n1.`user.id_str` = {user_id} RETURN count(r) as relationships, n2.`user.id_str` as neighbor')
    df = pd.DataFrame(a)
    try: 
        features["mentions"].iloc[i] = sum(df[0])
        features["neighbors"].iloc[i] = len(df)
    except:
        pass

C:\Users\xaris\AppData\Local\Temp\ipykernel_5200\284343195.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features["mentions"].iloc[i] = sum(df[0])
C:\Users\xaris\AppData\Local\Temp\ipykernel_5200\284343195.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features["neighbors"].iloc[i] = len(df)


In [67]:
from neo4j import GraphDatabase
import networkx as nx

driver = GraphDatabase.driver('bolt://localhost:7687', auth=("neo4j", "0000"))

In [68]:
#Store the out degree of each user

features["degreeOut"] = 0 
i=-1
for user_id in features["User_id"]:
    i += 1
    query =f"MATCH (u:User) WHERE u.`user.id_str` = {user_id} RETURN size((u)-->()) AS degreeOut"
    results = driver.session().run(query)
    df = pd.DataFrame(results)
    try: 
        features["degreeOut"].iloc[i] = df[0]
    except:
        pass

C:\Users\xaris\AppData\Local\Temp\ipykernel_5200\3051490355.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features["degreeOut"].iloc[i] = df[0]


In [72]:
features.to_csv('final_features.csv')